In [1]:
import hopsworks
import pandas as pd
import numpy as np

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://pocs.cloud.hopsworks.ai/p/125
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
input_transaction = {
    'account_id': '3b91e417a8584cdc56544cf372b8fe3f',
    'amount': 123,
    'category': 'Cash Withdrawal',
    'city': 'Collinwood',
    'latitude': 40.65538,
    'longitude': -74.38987
}

In [4]:
fraud_fv = fs.get_feature_view("fraud_model_fv", 1)

In [5]:
helpers_df = fraud_fv.get_inference_helper(entry={'account_id': input_transaction['account_id']})

In [23]:
def haversine(lat_1, long_1, lat_2, long_2):
    long_diff = long_1 - long_2
    lat_diff = lat_1 - lat_2

    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat_1) * np.cos(lat_2) * np.sin(long_diff/2.0)**2
    return 2*np.arcsin(np.sqrt(a + b))

In [32]:
passed_features = {
    "time_delta_t_minus_1": ((pd.Timestamp.now() - helpers_df["last_transaction_datetime"]) / np.timedelta64(1, 'D')).iloc[0],
    "loc_delta_t_minus_1": haversine(helpers_df['latitude'].iloc[0], helpers_df['longitude'].iloc[0], input_transaction['latitude'], input_transaction['longitude']),
    "outside_city": 0 if helpers_df['city'].iloc[0] == input_transaction['city'] else 1
}

In [36]:
passed_features.update(input_transaction)

In [38]:
feature_vector = fraud_fv.get_feature_vector(
                  entry={'account_id': input_transaction['account_id']},
                  passed_features=passed_features,
                  return_type="pandas",
            )


In [39]:
feature_vector

,amount,category,time_delta_t_minus_1,loc_delta_t_minus_1,outside_city,cc_provider,cc_type,age,cc_expiration_days,count,min_amount,max_amount,mean
0,123.0,0.0,13.623034,1.538847,0.0,1.0,1.0,57.690411,568.0,1.0,10.62,10.62,10.62
